# Understanding Your Customers: A Fresh Take on Analyzing Product Reviews

Aisha Al-Khaldi & Huda Joad

## Where are we in the data science pipeline?

- **Ask an interesting question**
- Get the data
- Explore the data
- Model the data
- Communicate/visualize the results

## Our Question

When customers want to express their thoughts and sentiments about a product, they usually do it through written reviews and assign a rating for their experience (in our case, a boolean of positive/negative). We would like to delve into the discussion of **the relationship between the content of the review and its assigned rating**.

We will be using Steam Web API, and try to perform a sentiment analysis on reviews to answer our question.

## Where are we in the data science pipeline?

- Ask an interesting question
- **Get the data**
- Explore the data
- Model the data
- Communicate/visualize the results

### Data Collection

In [1]:
import requests
import pandas as pd
import time
from requests.exceptions import ConnectionError
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Download necessary NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hudaj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hudaj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Steam has a web API. This means we don't actually need an API key to extract what we need, which are the reviews for various games. Instead, we can simply use different URLs.

More about Steam's web API can be found [here](https://partner.steamgames.com/doc/store/getreviews).

The web API allows us to filter by positive and negative reviews, which means we can use this to get labeled data without having to do it ourselves. Currently, there are over 1.6 million apps, although all of them do not necessarily have reviews.

In [2]:
url = "https://store.steampowered.com/appreviews/50?json=1&review_type=negative"
r = requests.get(url)
data = r.json()

data

{'success': 1,
 'query_summary': {'num_reviews': 2},
 'reviews': [{'recommendationid': '150673892',
   'author': {'steamid': '76561198355093882',
    'num_games_owned': 31,
    'num_reviews': 16,
    'playtime_forever': 541,
    'playtime_last_two_weeks': 254,
    'playtime_at_review': 541,
    'last_played': 1700597634},
   'language': 'english',
   'review': 'Well and truly, this game is pretty... okay. I say that cause the first half is fun, interesting, you get to see how one of the HECU would deal with the Black Mesa Incident nothing much to complain about there (except for buggy AI, but it\'s a half life 1 game. What else do you expect?).\nThe second half though just feels like its trying to be a harder half life 1. Race X aliens that are just sponges, with you taking insane amounts of damage and wasting a good chunk of ammo on a singular one. They\'re too fast, too spongy and a general nightmare to play against.\nThen you have what can only be described as Gearbox going "hmm wha

In this example we can see we get 2 negative reviews in JSON format. To get the reviews alone, let's try this.

In [3]:
[print(review['review']) for review in data['reviews']]

Well and truly, this game is pretty... okay. I say that cause the first half is fun, interesting, you get to see how one of the HECU would deal with the Black Mesa Incident nothing much to complain about there (except for buggy AI, but it's a half life 1 game. What else do you expect?).
The second half though just feels like its trying to be a harder half life 1. Race X aliens that are just sponges, with you taking insane amounts of damage and wasting a good chunk of ammo on a singular one. They're too fast, too spongy and a general nightmare to play against.
Then you have what can only be described as Gearbox going "hmm what if we made an annoying bit of half life (blast pit) even worse" and also going "what if we made a boss but terrible". It also feels like they were adding filler with how difficult and annoying the game gets.
The entire second half of the game is just not FUN. 

If you want more Half Life 1, get Blue Shift. It's short, but so much better.
The gameplay is great, but

[None, None]

We would know that these are negative reviews as we filtered for negative reviews in the URL for the Steam web API. With all this information, we can now programatically get labeled reviews. For the sake of this project, we will simply a minimum of the first 5,000 positive and negative reviews and go through the apps in order starting from app ID 1.

!!! Do not run the cell below. It was for generating the csv file, and will run for 6+ hours. You can skip this cell as we have already provided the csv for you.

In [4]:
# Function to fetch reviews from Steam API for a given game ID and review type (positive or negative)
def fetch_reviews(game_id, review_type):
    url = f"https://store.steampowered.com/appreviews/{game_id}?json=1&review_type={review_type}"
    try:
        r = requests.get(url)  # Make an HTTP GET request to the API
        if r.status_code == 200:
            data = r.json()  # Parse the JSON response
            return [review['review'] for review in data['reviews']]  # Return a list of reviews
    except ConnectionError:
        print(f"Connection error for game ID {game_id}. Retrying...")
        time.sleep(5)  # Wait for 5 seconds before retrying
        return fetch_reviews(game_id, review_type)  # Recursive retry
    return []  # Return an empty list if the request fails or an exception occurs

# Initialize lists to store positive and negative reviews
positive_reviews = []
negative_reviews = []
game_id = 1  # Start from the first game ID

# Loop until 5000 positive and 5000 negative reviews are collected
while len(positive_reviews) < 5000 or len(negative_reviews) < 5000:
    print(f"Game ID: {game_id}")
    if len(positive_reviews) < 5000:
        # Fetch and add positive reviews for the current game ID
        positive_reviews.extend(fetch_reviews(game_id, 'positive'))
        print(f"    Total Positive Reviews: {len(positive_reviews)}")

    if len(negative_reviews) < 5000:
        # Fetch and add negative reviews for the current game ID
        negative_reviews.extend(fetch_reviews(game_id, 'negative'))
        print(f"    Total Negative Reviews: {len(negative_reviews)}")

    game_id += 1  # Increment the game ID for the next iteration
    time.sleep(0.2)  # Pause for 0.5 seconds to avoid hitting the rate limit

# Combine the positive and negative reviews into a single list with their corresponding sentiment labels
reviews_data = [(review, 'positive') for review in positive_reviews] + \
               [(review, 'negative') for review in negative_reviews]

# Create a DataFrame from the combined review data
df = pd.DataFrame(reviews_data, columns=['review', 'sentiment'])

# Save the DataFrame to a CSV file, without the index and using UTF-8 encoding
df.to_csv('steam_reviews.csv', index=False, encoding='utf-8')

In [5]:
reviews_df = pd.read_csv('steam_reviews.csv', encoding='utf-8')

In [6]:
reviews_df

,review,sentiment
0,if ur tired of cs2 come back to 2000 and play ...,positive
1,Counter-Strike 1.6 was a significant part of m...,positive
2,rather pay for this than winrar,positive
3,Better then CS2,positive
4,"Every school that i have attended, had this in...",positive
...,...,...
9998,Its just a terrible clone of katamari. And IMO...,negative
9999,TL:DR this game was made 12 year old who got a...,negative
10000,Insultingly bad Katamari Damacy rip-off.,negative
10001,The poor man's Katamari Damacy.,negative


In [7]:
reviews_df.sentiment.value_counts()

negative    5003
positive    5000
Name: sentiment, dtype: int64

Now we have 10,003 total reviews from 15,970 apps. Providing an equal number of data for each label is important because ???.

## Where are we in the data science pipeline?

- Ask an interesting question
- Get the data
- **Explore the data**
- Model the data
- Communicate/visualize the results

### Data Processing

In [8]:
reviews_df.isna().sum()

review       5
sentiment    0
dtype: int64

Since we have enough reviews, we can simply drop the rows with null values.

In [9]:
reviews_df.dropna(inplace=True)
reviews_df.isna().sum()

review       0
sentiment    0
dtype: int64

In [10]:
reviews_df.shape

(9998, 2)

### Exploration/Visualization

In [11]:
# code

## Where are we in the data science pipeline?

- Ask an interesting question
- Get the data
- Explore the data
- **Model the data**
- Communicate/visualize the results

The model we will be using is a Random Forest Classifier. A random forest classifier can capture more complex relationships between words and sentiment. It's a good choice when you want to explore a slightly more advanced model without diving too deep into complex algorithms.

### Analysis/Machine Learning

In order to model the data, first we want to tokenize it, similar to what we did in assignment 3 in this course.

#### Text Preprocessing

In [12]:
# Create a new dataframe
reviews_preprocessed_df = reviews_df.copy()
reviews_preprocessed_df

,review,sentiment
0,if ur tired of cs2 come back to 2000 and play ...,positive
1,Counter-Strike 1.6 was a significant part of m...,positive
2,rather pay for this than winrar,positive
3,Better then CS2,positive
4,"Every school that i have attended, had this in...",positive
...,...,...
9998,Its just a terrible clone of katamari. And IMO...,negative
9999,TL:DR this game was made 12 year old who got a...,negative
10000,Insultingly bad Katamari Damacy rip-off.,negative
10001,The poor man's Katamari Damacy.,negative


In [13]:
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Remove punctuations and numbers
    text = re.sub('[^a-zA-Z]', ' ', text)
    # Remove single character words
    text = re.sub(r"\s+[a-zA-Z]\s+", ' ', text)
    # Remove multiple spaces
    text = re.sub(r'\s+', ' ', text)
    
    # Tokenization
    words = text.split()

    # Remove stopwords and Lemmatization
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words if word not in set(stopwords.words('english'))]
    
    return ' '.join(words) # The reason we join here is to use TfidfVectorizer later

# Apply preprocessing to each review
reviews_preprocessed_df['review'] = reviews_df['review'].apply(preprocess_text)

In [20]:
print(reviews_df.review.head())
print()
print(reviews_preprocessed_df.review.head())

0    if ur tired of cs2 come back to 2000 and play ...
1    Counter-Strike 1.6 was a significant part of m...
2                      rather pay for this than winrar
3                                      Better then CS2
4    Every school that i have attended, had this in...
Name: review, dtype: object

0                          ur tired c come back play c
1    counter strike significant part childhood firs...
2                                    rather pay winrar
3                                             better c
4       every school attended installed every computer
Name: review, dtype: object


#### Feature Extraction

In [15]:
# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(reviews_df['review']).toarray()

#### Split the data

In [16]:
y = reviews_df['sentiment'].map({'positive': 1, 'negative': 0}).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Train the Random Forest classifier

In [17]:
# Initialize the Random Forest Classifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

## Where are we in the data science pipeline?

- Ask an interesting question
- Get the data
- Explore the data
- Model the data
- **Communicate/visualize the results**

#### Evaluate the model

In [18]:
# Predicting the Test set results
y_pred = rf_classifier.predict(X_test)

# Evaluating the results
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

Accuracy: 0.7855
              precision    recall  f1-score   support

           0       0.75      0.84      0.79       968
           1       0.83      0.74      0.78      1032

    accuracy                           0.79      2000
   macro avg       0.79      0.79      0.79      2000
weighted avg       0.79      0.79      0.79      2000



#### Try using the model to predict the sentiment of new reviews

In [19]:
def predict_sentiment(review):
    review = preprocess_text(review)
    review_vector = tfidf_vectorizer.transform([review]).toarray()
    prediction = rf_classifier.predict(review_vector)
    return 'positive' if prediction[0] == 1 else 'negative'

# Example
print(predict_sentiment("This is a great product!"))

positive


### Insights

# Delete Later

Questions to answer
- Does the project clearly identify the problem? 
- Does the project clearly describe the relevant data or/and its collection? 
- Does the project clearly explain how the data can be used to draw conclusions about the 
- underlying system? 
- Does the report clearly explain the work that was done? 
- Is the project innovative or novel? 
- Is the model built accurate enough? 
- Does the project use techniques presented in the course (or clearly related to topics covered in the course) to understand and analyze the data for this problem? 
- Does the report explain how this work fits around related work in this subject area? 
- Does the report provide directions for further investigation?